<a href="https://colab.research.google.com/github/autofollower350/Zzzzzzzz/blob/main/ZZZZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import subprocess
import os

# ✅ Check if running in Google Colab
try:
    from google.colab import output
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# ✅ Auto-Install Required Modules
def install_modules():
    modules = ["yt-dlp", "pyrogram", "tgcrypto", "ffmpeg-python"]

    if IN_COLAB:
        print("🔄 Running in Google Colab, Installing Required Modules...")
        subprocess.run(["apt-get", "update"], check=True)
        subprocess.run(["apt-get", "install", "-y", "ffmpeg"], check=True)  # Install FFmpeg
        subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True)

    for module in modules:
        try:
            __import__(module)
        except ImportError:
            print(f"⚡ Installing {module}...")
            subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", module], check=True)

    # ✅ Google Colab में sys.path को अपडेट करें
    if IN_COLAB:
        sys.path.append("/usr/local/lib/python3.10/dist-packages/")

install_modules()

# ✅ अब yt-dlp और बाकी पैकेज लोड होंगे
import yt_dlp
import ffmpeg
from pyrogram import Client, filters
from pyrogram.errors import FloodWait

print("✅ All modules installed and loaded successfully!")

In [ ]:
import sys
import subprocess
import nest_asyncio
import asyncio
import os
import re
import yt_dlp
import ffmpeg
from pyrogram import Client, filters
from pyrogram.errors import FloodWait

# ✅ Check if running in Google Colab
try:
    from google.colab import output
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# ✅ Auto-Install Required Modules
def install_modules():
    modules = ["yt-dlp", "pyrogram", "tgcrypto", "ffmpeg-python"]

    if IN_COLAB:
        print("🔄 Running in Google Colab, Installing Required Modules...")
        subprocess.run(["apt-get", "update"], check=True)
        subprocess.run(["apt-get", "install", "-y", "ffmpeg"], check=True)  # Install FFmpeg
        subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True)

    for module in modules:
        try:
            __import__(module)
        except ImportError:
            print(f"⚡ Installing {module}...")
            subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", module], check=True)

install_modules()

nest_asyncio.apply()

# ✅ Telegram Bot API Keys
API_ID = "28590286"
API_HASH = "6a68cc6b41219dc57b7a52914032f92f"
BOT_TOKEN = "7412939071:AAFk6_MJziu22UnZOv9crH-HcwLDqzQ1SVg"

app = Client("m3u8_video_bot", api_id=API_ID, api_hash=API_HASH, bot_token=BOT_TOKEN)

THUMBNAIL_PATH = "thumbnail.jpg"

# ✅ Default Caption for All Videos
DEFAULT_CAPTION = "🎬 {video_name}\n🔥 𝐸𝓍𝓉𝓇𝒶𝒸𝓉𝑒𝒹 𝐵𝓎 : 𝑅𝒶𝓉𝒽𝑜𝓇𝑒"

# ✅ Extract Name and URL Function
def extract_m3u8_names_and_urls(text):
    m3u8_pattern = re.compile(r"https?://\S+\.m3u8")
    name_url_dict = {}

    for line in text.split("\n"):
        match = m3u8_pattern.search(line)
        if match:
            url = match.group()
            name = line[:match.start()].strip()
            name_url_dict[name] = url

    return name_url_dict

@app.on_message(filters.command("start"))
async def start(client, message):
    await message.reply_text("Send me a .txt file containing M3U8 video links, and I'll download and upload them one by one!")

@app.on_message(filters.photo & filters.private)
async def save_thumbnail(client, message):
    global THUMBNAIL_PATH
    original_thumbnail = await message.download()
    THUMBNAIL_PATH = "thumbnail.jpg"

    # ✅ Resize Thumbnail to 1280x720 using FFmpeg
    ffmpeg.input(original_thumbnail).output(THUMBNAIL_PATH, vf="scale=1280:720", format="image2").run(overwrite_output=True)

    await message.reply_text("✅ Thumbnail saved successfully in full-screen format!")

@app.on_message(filters.command("delthumb") & filters.private)
async def delete_thumbnail(client, message):
    global THUMBNAIL_PATH
    if os.path.exists(THUMBNAIL_PATH):
        os.remove(THUMBNAIL_PATH)
        THUMBNAIL_PATH = None
        await message.reply_text("🗑️ Thumbnail deleted successfully!")
    else:
        await message.reply_text("⚠️ No thumbnail found to delete.")

@app.on_message(filters.command("showthumb") & filters.private)
async def show_thumbnail(client, message):
    global THUMBNAIL_PATH
    if THUMBNAIL_PATH and os.path.exists(THUMBNAIL_PATH):
        await message.reply_photo(THUMBNAIL_PATH, caption="🖼️ Your current thumbnail.")
    else:
        await message.reply_text("⚠️ No thumbnail is set!")

@app.on_message(filters.document & filters.private)
async def download_m3u8_from_txt(client, message):
    if not message.document.file_name.endswith(".txt"):
        return await message.reply_text("Please send a .txt file containing M3U8 video links!")

    msg = await message.reply_text("📂 Processing file...")

    file_bytes = await client.download_media(message.document, in_memory=True)

    try:
        text_content = file_bytes.getvalue().decode("utf-8").strip()
        m3u8_dict = extract_m3u8_names_and_urls(text_content)

        if not m3u8_dict:
            return await message.reply_text("❌ No valid M3U8 video links found in the file!")

        await msg.edit_text(f"✅ Found {len(m3u8_dict)} videos. Downloading and uploading one by one...")

        for index, (name, m3u8_url) in enumerate(m3u8_dict.items(), 1):
            await download_and_upload(client, message.chat.id, m3u8_url, name, msg)

        await msg.edit_text("✅ All M3U8 videos processed!")

    except Exception as e:
        await message.reply_text(f"❌ Error: {str(e)}")

# 🔹 yt-dlp से M3U8 डाउनलोड और अपलोड करने का फंक्शन
async def download_and_upload(client, chat_id, m3u8_url, video_name, msg):
    temp_video_path = f"{video_name}.mp4"

    await msg.edit_text(f"🔄 Downloading {video_name} using yt-dlp...")

    ydl_opts = {
        "format": "best",
        "outtmpl": temp_video_path,
        "hls_prefer_native": True,
        "fixup": "never",
        "skip_unavailable_fragments": True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([m3u8_url])
    except Exception as e:
        await msg.edit_text(f"❌ Failed to download {video_name}: {str(e)}")
        return False

    await msg.edit_text(f"📤 Uploading {video_name}...")

    try:
        await fast_upload(client, chat_id, temp_video_path, video_name, msg)
        os.remove(temp_video_path)
    except Exception as e:
        await msg.edit_text(f"❌ Upload failed: {str(e)}")
        return False

# ✅ Progress Bar Function
def progress_bar(percent):
    bar_length = 10
    filled = int(bar_length * percent / 100)
    return "▰" * filled + "▱" * (bar_length - filled)

# ✅ Progress Update & Upload Function
async def fast_upload(client, chat_id, file_path, video_name, msg):
    async def upload_progress(current, total):
        percent = round((current / total) * 100, 2)
        bar = progress_bar(percent)
        try:
            await msg.edit_text(f"📤 Uploading: {bar} {percent}%")
        except:
            pass  # FloodWait Issue Ignore

    try:
        await client.send_video(
            chat_id,
            file_path,
            caption=DEFAULT_CAPTION.format(video_name=video_name),
            thumb=THUMBNAIL_PATH if os.path.exists(THUMBNAIL_PATH) else None,
            supports_streaming=True,
            progress=upload_progress
        )
        await msg.edit_text("✅ Upload completed!")

    except FloodWait as e:
        wait_time = e.value
        print(f"⏳ FloodWait: Sleeping for {wait_time} seconds...")
        await asyncio.sleep(wait_time)
        await fast_upload(client, chat_id, file_path, video_name, msg)

# ✅ बॉट को स्टार्ट करो
async def start_bot():
    await app.start()
    print("✅ Bot is running...")
    await asyncio.Event().wait()

loop = asyncio.get_event_loop()
loop.run_until_complete(start_bot())